In [4]:
import logging, logging.handlers
import sys
import pickle
import numpy as np
from collections import namedtuple
from itertools import count
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable

import seaborn as sns; sns.set(color_codes=True)
%matplotlib inline
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import math

# round_ = 1
# lookahead = 1
target_update = 10000
w = 10
h = 10
life_cost = 0.05
# env_name = 'Grid'
# env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)
# logger = logging.getLogger()
# file_name = './data/results_%s.log' %(env_name)
# fh = logging.handlers.RotatingFileHandler(file_name)
# fh.setLevel(logging.DEBUG)#no matter what level I set here
# formatter = logging.Formatter('%(asctime)s:%(message)s')
# fh.setFormatter(formatter)
# logger.addHandler(fh)

In [5]:
batchsize = 64
USE_CUDA = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
seed = 0
number_action = 4
dimension = 2

import torch
import torch.nn as nn
import torch.nn.functional as F
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv' or 'SNConv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
class DQN(nn.Module):
    def __init__(self, dimension=2, num_actions=number_action):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(dimension, 50)
#         self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(50, num_actions)
    def forward(self, x):
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
#         x = F.relu(self.fc2(x.view(x.size(0), -1)))
        return self.fc3(x)


In [6]:
w_ = w-1
h_ = 0
h__ = h-1
gamma = 0.99
pos_rew = 1
def recurs(imagine_,h,Q):
    if h == 0:
        val = Q(imagine_.unsqueeze(0).unsqueeze(0).type(dtype)).max(1)
        return val[0], val[1][0]
    else:
        action = 0
        for i in range(number_action):
            imagine = torch.clone(imagine_)
            if i == 0:
                imagine[1] = torch.clone(torch.clamp(imagine_[1] - 1,0,h-1))
                cum_return_ = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                               - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                
            if i == 1:
                imagine[0] = torch.clone(torch.clamp(imagine_[0] - 1,0,w-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 1
            if i == 2:
                imagine[1] = torch.clone(torch.clamp(imagine_[1] + 1,0,h-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 2
            if i == 3:
                imagine[0] = torch.clone(torch.clamp(imagine_[0] + 1,0,w-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 3   
#         print(cum_return_)            
        return cum_return_, torch.tensor(action)
                         

In [ ]:
#action
#  0
#1   3
#  2
#         w
# x=0,y=0    x=w,y=0    
#                      h
# x=0,y=h    x=w,y=h   

for round_count in range(10):
    round_ = round_count + 1
    lookahead = 1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)

    init_w = 0
    init_h = h-1
    tot_epis = 1000
    epsilon = .1


    Q = DQN(dimension,number_action).type(dtype) 
    Target_Q = DQN(dimension,number_action).type(dtype) 
    Q.apply(weights_init)
    optimizer = optim.SGD(Q.parameters(), lr = .0001, momentum=0.9)


    actions = torch.zeros((1),dtype=torch.int8)
    states = torch.zeros((1,2),dtype=torch.int8)
    # done = torch.zeros((1)).type(dtype) 
    # states = torch.tensor([init_w,init_h])
    rewards = torch.zeros((1)).type(dtype) 
    tot_rew = torch.zeros((1)).type(dtype) 
    # next_states = torch.zeros((1,2),dtype=torch.int8)
    e = 0
    tot_time = 2100000
    time = 0
    while time < tot_time:
        t = 0
        goal = 0
        cum_rew = 0
        # Initiate the state
        if time > 0:
            states = torch.cat((states,torch.tensor([[0,0]],dtype=torch.int8)),0)
        state = torch.clone(states[-1])
        while t<100 and goal==0:
            time = time+1
            next_state =  torch.clone(state)
            sample = random.random()
            if sample < epsilon:
                action = torch.tensor(random.randrange(number_action))
            else:
                action = recurs(state,lookahead,Q)[1]
            if time == 1:
                actions[0] = action.unsqueeze(0).type(dtype) 
            else:
                actions = torch.cat((actions,action.unsqueeze(0).type(torch.int8)),0)
            reward = pos_rew*torch.all(torch.eq(state, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                    - torch.all(torch.eq(state[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
    #         print(state)
            if reward != -1 * life_cost :
                goal = 1
#                 if reward == pos_rew - life_cost:
#                     print('bingo:t:',t,state)

            if time == 1:
                rewards[0] = reward.unsqueeze(0)
            else:
                rewards = torch.cat((rewards,reward.unsqueeze(0)),0)
            if action == 0:
                next_state[1] = torch.clone(torch.clamp(state[1] - 1,0,h-1))
            if action == 1:
                next_state[0] = torch.clone(torch.clamp(state[0] - 1,0,w-1))
            if action == 2:
                next_state[1] = torch.clone(torch.clamp(state[1] + 1,0,h-1))
            if action == 3:
                next_state[0] = torch.clone(torch.clamp(state[0] + 1,0,w-1))
    #         print(state.type(torch.int).numpy(),action.type(torch.int).numpy(),next_state.type(torch.int).numpy()) 
            cum_rew = cum_rew + gamma * torch.clone(reward)   
            states = torch.cat((states,torch.clone(next_state.unsqueeze(0))),0)
            state = torch.clone(next_state)


    #         current_value = Q(state.unsqueeze(0).unsqueeze(0).type(dtype))[0][action]    
    #         loss = current_value-reward.type(dtype)-(1-goal)*gamma*Target_Q(next_state.unsqueeze(0).unsqueeze(0).type(dtype)).max(1)[0].detach()
    #         d_error = loss.clamp(-1, 1)[0]
    #         optimizer.zero_grad()
    #         current_value.backward(d_error.data)
    #         optimizer.step()
            if time>5000:
                if time % 4 == 0:        
                    choices = np.random.choice(time, batchsize)
                    current_action = actions[choices].unsqueeze(1)
                    current_rewards = rewards[choices].type(dtype)
                    indecies = np.linspace(0,batchsize-1,batchsize,dtype=np.int32)
                    current_nextstates = states[choices+1].unsqueeze(1).type(dtype)
                    current_value = Q(states[choices].unsqueeze(1).type(dtype))[indecies,current_action.squeeze(1).type(torch.int32).numpy()]
                    loss = current_value-current_rewards-(current_rewards== -1 * life_cost).type(dtype)*gamma*Target_Q(current_nextstates).max(1)[0].detach()
                    d_error = loss.clamp(-1, 1)
                    optimizer.zero_grad()
                    current_value.backward(d_error.data)
                    optimizer.step()
            t = t+1

            if time % target_update == 0:
                Target_Q.load_state_dict(Q.state_dict())


        if len(tot_rew) == 1:
            tot_rew[0] = cum_rew.unsqueeze(0).type(dtype)
        else:
            tot_rew = torch.cat((tot_rew,cum_rew.unsqueeze(0).type(dtype)),0) 
        if e%100==0:    
            print('e:',e,'t:',time,'rew:',cum_rew.type(dtype).numpy())
        e = e + 1
    #     s = torch.zeros((1,2),dtype=torch.int32)
    #     s[0,0] = 3
    #     s[0,1] = 3
    #     ss = torch.zeros((1,2),dtype=torch.int32)
    #     ss[0,0] = 0
    #     ss[0,1] = 0
    #     print(Q(s.type(dtype))-Q(ss.type(dtype)))


    fnam = './data/reward%s' %(env_name)
    np.save(fnam,rewards.numpy())                          




    #                 for i2 in range(number_action):







    #  0
    #1   3
    #  2 

e: 0 t: 12 rew: 0.3960001
e: 100 t: 1170 rew: 0.44550008
e: 200 t: 2315 rew: 0.3465001
e: 300 t: 3435 rew: 0.2970001
e: 400 t: 4558 rew: 0.49500006
e: 500 t: 6102 rew: 0.3465001
e: 600 t: 7237 rew: 0.44550008
e: 700 t: 8385 rew: 0.49500006
e: 800 t: 9557 rew: 0.3960001
e: 900 t: 12663 rew: 0.3960001
e: 1000 t: 16065 rew: 0.49500006
e: 1100 t: 19732 rew: -0.49499977
e: 1200 t: 24286 rew: -4.9499993
e: 1300 t: 29714 rew: -0.98999965
e: 1400 t: 36084 rew: 0.49500006
e: 1500 t: 41058 rew: -2.8709993
e: 1600 t: 46082 rew: -4.9499993
e: 1700 t: 50064 rew: -3.0194993
e: 1800 t: 55328 rew: 0.19800013
e: 1900 t: 59521 rew: -2.7719994
e: 2000 t: 67065 rew: 0.3960001
e: 2100 t: 70010 rew: 0.3465001
e: 2200 t: 71625 rew: 0.49500006
e: 2300 t: 73435 rew: 0.49500006
e: 2400 t: 75395 rew: -3.2669995
e: 2500 t: 76647 rew: 0.49500006
e: 2600 t: 78206 rew: 0.49500006
e: 2700 t: 79495 rew: 0.49500006
e: 2800 t: 81377 rew: -4.9499993
e: 2900 t: 83466 rew: 0.49500006
e: 3000 t: 85694 rew: 0.49500006
e: 310

e: 23900 t: 844309 rew: -4.9499993
e: 24000 t: 849274 rew: 0.24750012
e: 24100 t: 855114 rew: -4.9499993
e: 24200 t: 862926 rew: -4.9499993
e: 24300 t: 870823 rew: -0.8909997
e: 24400 t: 879110 rew: -4.9499993
e: 24500 t: 888093 rew: -3.7124994
e: 24600 t: 895623 rew: -3.7124994
e: 24700 t: 901503 rew: -0.3464998
e: 24800 t: 907354 rew: -4.9499993
e: 24900 t: 911430 rew: 0.2970001
e: 25000 t: 914076 rew: 0.3465001
e: 25100 t: 920255 rew: 0.49500006
e: 25200 t: 921624 rew: 0.49500006
e: 25300 t: 923262 rew: 0.3465001
e: 25400 t: 925646 rew: 0.44550008
e: 25500 t: 927163 rew: 0.49500006
e: 25600 t: 929844 rew: 0.2970001
e: 25700 t: 932290 rew: -2.0294995
e: 25800 t: 938336 rew: -4.9499993
e: 25900 t: 946727 rew: 0.44550008
e: 26000 t: 953750 rew: -2.5244994
e: 26100 t: 958760 rew: -0.49499977
e: 26200 t: 964075 rew: 0.49500006
e: 26300 t: 970546 rew: 0.3960001
e: 26400 t: 978663 rew: -4.9499993
e: 26500 t: 988619 rew: -4.9499993
e: 26600 t: 998619 rew: -4.9499993
e: 26700 t: 1008619 rew:

e: 46800 t: 1836369 rew: -4.9499993
e: 46900 t: 1845671 rew: -4.9499993
e: 47000 t: 1855109 rew: -4.9499993
e: 47100 t: 1863504 rew: -4.9499993
e: 47200 t: 1872087 rew: -1.2869997
e: 47300 t: 1880114 rew: -4.9499993
e: 47400 t: 1883954 rew: -0.59399974
e: 47500 t: 1885362 rew: 0.3960001
e: 47600 t: 1888730 rew: -0.98999965
e: 47700 t: 1892219 rew: -3.2669995
e: 47800 t: 1895998 rew: -0.94049966
e: 47900 t: 1900249 rew: -0.8909997
e: 48000 t: 1905210 rew: -4.9499993
e: 48100 t: 1909333 rew: 0.2970001
e: 48200 t: 1915145 rew: -4.9499993
e: 48300 t: 1921013 rew: 0.14850014
e: 48400 t: 1925573 rew: -4.9499993
e: 48500 t: 1933938 rew: -4.9499993
e: 48600 t: 1943330 rew: -4.9499993
e: 48700 t: 1953048 rew: -4.9499993
e: 48800 t: 1962623 rew: -4.9499993
e: 48900 t: 1972235 rew: -4.9499993
e: 49000 t: 1982072 rew: -4.9499993
e: 49100 t: 1989255 rew: -4.9499993
e: 49200 t: 1995045 rew: -0.98999965
e: 49300 t: 2000921 rew: -4.9499993
e: 49400 t: 2004187 rew: -4.9499993
e: 49500 t: 2007469 rew: -

e: 19100 t: 681027 rew: 0.49500006
e: 19200 t: 685204 rew: -0.3464998
e: 19300 t: 688829 rew: 0.49500006
e: 19400 t: 691981 rew: 0.3465001
e: 19500 t: 697003 rew: 0.49500006
e: 19600 t: 701570 rew: -4.9499993
e: 19700 t: 709655 rew: -4.9499993
e: 19800 t: 718194 rew: -4.9499993
e: 19900 t: 723726 rew: -4.9499993
e: 20000 t: 730113 rew: -3.8114994
e: 20100 t: 736007 rew: -4.9499993
e: 20200 t: 740502 rew: 0.49500006
e: 20300 t: 746265 rew: 0.49500006
e: 20400 t: 752110 rew: -4.9499993
e: 20500 t: 761566 rew: -4.9499993
e: 20600 t: 771011 rew: -4.9499993
e: 20700 t: 780796 rew: -4.9499993
e: 20800 t: 789541 rew: -1.5344996
e: 20900 t: 797373 rew: -0.59399974
e: 21000 t: 806193 rew: -3.9599993
e: 21100 t: 812876 rew: 0.2970001
e: 21200 t: 817052 rew: -1.4849997
e: 21300 t: 822388 rew: 0.49500006
e: 21400 t: 828096 rew: -4.9499993
e: 21500 t: 835539 rew: -2.2769995
e: 21600 t: 841171 rew: 0.44550008
e: 21700 t: 844893 rew: 0.49500006
e: 21800 t: 848868 rew: 0.49500006
e: 21900 t: 855679 re

e: 42100 t: 1735588 rew: -4.9499993
e: 42200 t: 1745169 rew: -4.9499993
e: 42300 t: 1754824 rew: -4.9499993
e: 42400 t: 1762315 rew: -0.7919997
e: 42500 t: 1765063 rew: 0.49500006
e: 42600 t: 1769206 rew: -4.9499993
e: 42700 t: 1772199 rew: 0.49500006
e: 42800 t: 1773882 rew: 0.49500006
e: 42900 t: 1775979 rew: 0.2970001
e: 43000 t: 1777888 rew: 0.3960001
e: 43100 t: 1780491 rew: -1.4354997
e: 43200 t: 1784992 rew: -4.9499993
e: 43300 t: 1788769 rew: 0.3960001
e: 43400 t: 1792161 rew: 0.3960001
e: 43500 t: 1795968 rew: 0.099000156
e: 43600 t: 1799791 rew: -0.19799984
e: 43700 t: 1801312 rew: 0.3960001
e: 43800 t: 1802716 rew: 0.3960001
e: 43900 t: 1804009 rew: 0.3960001
e: 44000 t: 1805249 rew: 0.49500006
e: 44100 t: 1806379 rew: 0.49500006
e: 44200 t: 1807529 rew: 0.49500006
e: 44300 t: 1809865 rew: -0.44549978
e: 44400 t: 1812324 rew: 0.3960001
e: 44500 t: 1814232 rew: 0.49500006
e: 44600 t: 1817453 rew: 0.44550008
e: 44700 t: 1819022 rew: 0.3465001
e: 44800 t: 1820711 rew: 0.2970001

e: 16800 t: 426275 rew: -4.9499993
e: 16900 t: 433794 rew: -4.9499993
e: 17000 t: 440670 rew: -4.9499993
e: 17100 t: 447522 rew: -3.7124994
e: 17200 t: 451577 rew: 1.7881393e-07
e: 17300 t: 453571 rew: -4.9499993
e: 17400 t: 455958 rew: 0.3960001
e: 17500 t: 459683 rew: -4.9499993
e: 17600 t: 461548 rew: 0.49500006
e: 17700 t: 462750 rew: 0.49500006
e: 17800 t: 463933 rew: 0.49500006
e: 17900 t: 465652 rew: 0.49500006
e: 18000 t: 468642 rew: 0.49500006
e: 18100 t: 470522 rew: 0.24750012
e: 18200 t: 472014 rew: 0.14850014
e: 18300 t: 473537 rew: 0.49500006
e: 18400 t: 475212 rew: 0.44550008
e: 18500 t: 477684 rew: 0.2970001
e: 18600 t: 479690 rew: 0.44550008
e: 18700 t: 481253 rew: -1.7324996
e: 18800 t: 483405 rew: 0.49500006
e: 18900 t: 485318 rew: 0.3960001
e: 19000 t: 487562 rew: -0.59399974
e: 19100 t: 490845 rew: -0.6929997
e: 19200 t: 494093 rew: 0.24750012
e: 19300 t: 497522 rew: -1.8809996
e: 19400 t: 500345 rew: -0.04949981
e: 19500 t: 504826 rew: -3.1679995
e: 19600 t: 508590

e: 40000 t: 1429791 rew: 0.2970001
e: 40100 t: 1431209 rew: 0.49500006
e: 40200 t: 1433248 rew: 0.44550008
e: 40300 t: 1434723 rew: 0.44550008
e: 40400 t: 1436353 rew: 0.49500006
e: 40500 t: 1438431 rew: 0.49500006
e: 40600 t: 1440334 rew: 0.49500006
e: 40700 t: 1442343 rew: 0.3465001
e: 40800 t: 1443915 rew: 0.3960001
e: 40900 t: 1445731 rew: 0.49500006
e: 41000 t: 1451254 rew: -4.9499993
e: 41100 t: 1460933 rew: -4.9499993
e: 41200 t: 1470928 rew: -4.9499993
e: 41300 t: 1480682 rew: -4.9499993
e: 41400 t: 1485628 rew: -0.04949981
e: 41500 t: 1490626 rew: 0.49500006
e: 41600 t: 1492059 rew: -4.9499993
e: 41700 t: 1493303 rew: 0.49500006
e: 41800 t: 1494478 rew: 0.49500006
e: 41900 t: 1496666 rew: -0.94049966
e: 42000 t: 1500612 rew: -4.9499993
e: 42100 t: 1507855 rew: 0.3960001
e: 42200 t: 1511008 rew: 0.49500006
e: 42300 t: 1513170 rew: 0.44550008
e: 42400 t: 1517207 rew: 0.2970001
e: 42500 t: 1520685 rew: -4.9499993
e: 42600 t: 1525414 rew: -3.7619994
e: 42700 t: 1531571 rew: -1.831

e: 9800 t: 300189 rew: 0.3960001
e: 9900 t: 301961 rew: 0.24750012
e: 10000 t: 304201 rew: 0.3960001
e: 10100 t: 305858 rew: 0.3960001
e: 10200 t: 308074 rew: -0.09899986
e: 10300 t: 310418 rew: 0.44550008
e: 10400 t: 312127 rew: 0.3465001
e: 10500 t: 313836 rew: -3.9599993
e: 10600 t: 316034 rew: 0.49500006
e: 10700 t: 318285 rew: -1.2869997
e: 10800 t: 320777 rew: -1.8809996
e: 10900 t: 324182 rew: 0.44550008
e: 11000 t: 329434 rew: -4.9499993
e: 11100 t: 334617 rew: 0.44550008
e: 11200 t: 339074 rew: -4.9499993
e: 11300 t: 344533 rew: -4.9499993
e: 11400 t: 348296 rew: -4.9499993
e: 11500 t: 350650 rew: 0.2970001
e: 11600 t: 353177 rew: 0.49500006
e: 11700 t: 354728 rew: 0.3960001
e: 11800 t: 356746 rew: -1.0889997
e: 11900 t: 359550 rew: 0.2970001
e: 12000 t: 361006 rew: 0.49500006
e: 12100 t: 362672 rew: 0.49500006
e: 12200 t: 364245 rew: 0.49500006
e: 12300 t: 366400 rew: -3.1679995
e: 12400 t: 368007 rew: 0.44550008
e: 12500 t: 369722 rew: 0.49500006
e: 12600 t: 372214 rew: -3.2

e: 33300 t: 1050987 rew: -4.9499993
e: 33400 t: 1056580 rew: 0.3465001
e: 33500 t: 1063719 rew: -4.9499993
e: 33600 t: 1073027 rew: -4.9499993
e: 33700 t: 1081564 rew: -4.9499993
e: 33800 t: 1090152 rew: -4.9499993
e: 33900 t: 1097415 rew: -4.9499993
e: 34000 t: 1106783 rew: -4.9499993
e: 34100 t: 1116231 rew: -2.3759995
e: 34200 t: 1124518 rew: -4.9499993
e: 34300 t: 1133951 rew: -4.9499993
e: 34400 t: 1143399 rew: -4.9499993
e: 34500 t: 1153250 rew: -4.9499993
e: 34600 t: 1162988 rew: -4.9499993
e: 34700 t: 1172081 rew: -4.9499993
e: 34800 t: 1181779 rew: -3.0194993
e: 34900 t: 1191520 rew: -4.9499993
e: 35000 t: 1201179 rew: -4.9499993
e: 35100 t: 1210730 rew: -4.9499993
e: 35200 t: 1215047 rew: 0.49500006
e: 35300 t: 1217730 rew: 0.49500006
e: 35400 t: 1219886 rew: 0.3960001
e: 35500 t: 1227954 rew: 0.49500006
e: 35600 t: 1230974 rew: 0.49500006
e: 35700 t: 1234732 rew: -4.9499993
e: 35800 t: 1240824 rew: 0.3960001
e: 35900 t: 1242785 rew: 0.14850014
e: 36000 t: 1244830 rew: 0.4950

In [ ]:
bandwidth = 100
sampling = 1000
# tot_time = 100000

LA0 = np.zeros((2,int(tot_time/sampling )-bandwidth))
lookahead = 0
for r in range(2):
    round_ = r+1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)

    fnam = './data/reward%s.npy' %(env_name)
    tot_3 = np.load(fnam)
    total_3_sample= np.zeros(int(tot_time/sampling ))
    total_3= np.zeros(int(tot_time/sampling )-bandwidth)
    for i in range(int(tot_time/sampling)):
        total_3_sample[i] = np.sum(tot_3[sampling*i:sampling*i+sampling])/sampling
    for i in range(len(total_3_sample)-bandwidth):
        total_3[i] = np.sum(total_3_sample[i:i+bandwidth])/bandwidth

    LA0[r,:] = total_3


In [ ]:
LA1 = np.zeros((1,int(tot_time/sampling )-bandwidth))
lookahead = 1
for r in range(1):
    round_ = r+1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)
    fnam = './data/reward%s.npy' %(env_name)
    tot_3 = np.load(fnam)
    total_3_sample= np.zeros(int(tot_time/sampling ))
    total_3= np.zeros(int(tot_time/sampling )-bandwidth)
    for i in range(int(tot_time/sampling)):
        total_3_sample[i] = np.sum(tot_3[sampling*i:sampling*i+sampling])/sampling
    for i in range(len(total_3_sample)-bandwidth):
        total_3[i] = np.sum(total_3_sample[i:i+bandwidth])/bandwidth

    LA1[r,:] = total_3



In [ ]:
fonts = 15
fram = np.linspace(1,tot_time,int(tot_time/sampling ),dtype=np.int32)[0:-bandwidth]
plt.ticklabel_format(style='sci', axis='x')
plt.ticklabel_format(axis='both', style='sci', scilimits=(-2,2))
plt.legend(fontsize=fonts)

# ax = sns.tsplot(data=LA0,time=fram,value='Value',color = 'b',condition = "LA = 0")
ax = sns.tsplot(data=LA1,time=fram,value='Value',color = 'c',condition = "LA = 1")

plt.xlabel("Number of steps",fontsize=fonts, family = 'serif')
plt.ylabel("Average reward per episode",fontsize=fonts, family = 'serif')
plt.title("%s" %(env_name),fontsize=fonts, family = 'serif')
# ax.set(xlabel='Number of steps', ylabel='Average reward per episode')

# plt.savefig("Pong_reward_lambda_3_runs.png",format='png', dpi=2000)
# plt.savefig("Pong_reward_lambda_3_runs.eps",format='eps', dpi=2000)

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 3
s[0,1] = 3
ss = torch.zeros((1,2),dtype=torch.int32)
ss[0,0] = 0
ss[0,1] = 0
print(Q(s.type(dtype))-Q(ss.type(dtype)))

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 3
s[0,1] = 3
Q(s.type(dtype))

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 0
s[0,1] = 0
Q(s.type(dtype))

In [ ]:
current_nextstates= states[choices+1].unsqueeze(1).type(dtype)
Target_Q(current_nextstates).max(1)[0].detach()

In [ ]:
d_error

In [ ]:
np.linspace(0,batchsize-1,batchsize,dtype=np.int32)

In [ ]:
rewards

In [ ]:
np.linspace(0,batchsize-1,batchsize,dtype=np.int32)

In [ ]:
current_value

In [ ]:
current_rewards

In [ ]:
(current_rewards==0)*gamma*Target_Q(current_nextstates).max(1)[0].detach()

In [ ]:
rewards

In [ ]:
# current_rewards

In [ ]:
current_rewards[0]==life_cost

In [ ]:
current_rewards == -1 * life_cost

In [ ]:
current_rewards[0]

In [ ]:
choices = np.random.choice(time, batchsize)
current_action = actions[choices].unsqueeze(1)
current_rewards = rewards[choices].type(dtype)
indecies = np.linspace(0,batchsize-1,batchsize,dtype=np.int32)
current_nextstates = states[choices+1].unsqueeze(1).type(dtype)
current_value = Q(states[choices].unsqueeze(1).type(dtype))[indecies,current_action.squeeze(1).type(torch.int32).numpy()]
loss = current_value-current_rewards-(current_rewards== -1 * life_cost).type(dtype)*gamma*Target_Q(current_nextstates).max(1)[0].detach()
d_error = loss.clamp(-1, 1)
optimizer.zero_grad()
current_value.backward(d_error.data)
# optimizer.step()

In [ ]:
d_error.shape